In [5]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

In [4]:
#main code



# "dummy url"
dummy_url="https://www.amazon.{country}/dp/{asin}"

#get the csv file which give you idea that which urls are working properly

def main_url(country,Asin):
    return f"https://www.amazon.{country}/dp/{Asin}"

#call the csv file where all the counttry and Asin domains are availabel

df=pd.read_excel("file.xlsx",usecols=["Asin","country"])


list_url=[]
for country,Asin in zip(df.country.values,df.Asin.values):
    list_url.append(main_url(country,Asin))


df["url"]=list_url

df





,Asin,country,url
0,1015.0,de,https://www.amazon.de/dp/1015.0
1,1015.0,fr,https://www.amazon.fr/dp/1015.0
2,000004458X,de,https://www.amazon.de/dp/000004458X
3,000004458X,fr,https://www.amazon.fr/dp/000004458X
4,1002198.0,de,https://www.amazon.de/dp/1002198.0
...,...,...,...
995,4484924.0,fr,https://www.amazon.fr/dp/4484924.0
996,4485742.0,de,https://www.amazon.de/dp/4485742.0
997,4485742.0,fr,https://www.amazon.fr/dp/4485742.0
998,4486072.0,de,https://www.amazon.de/dp/4486072.0


In [24]:
import json
import time

In [25]:
#header


headers={
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36 Edg/103.0.1264.77"
}



def scrape(url):
    product_details=[]
    
    
    try:
        page=requests.get(url,headers=headers)
        code=status_checker(page.status_code)
        soup=BeautifulSoup(page.content,"html.parser")
        title=soup.find(id="productTitle").get_text().strip()
        img_link=soup.find("div",id="imageBlockContainer").find("img")["src"]
        price=soup.find("ul",class_="a-unordered-list a-nostyle a-button-list a-horizontal").find("span",class_="a-color-base").get_text().strip()
        #product_details
        parent=soup.find("body").find("div",id="detailBullets_feature_div").find("ul").find_all("li")
        
        
        for i in parent:
            product_details.append(re.split("\u200e",i.text.replace(" ","").replace(":","").replace("\n",""),2)[1])
            

        
        
    except:
        img_link=soup.find("div",id="imgTagWrapperId").find("img")["src"]
        price=soup.find("div",class_="a-section a-spacing-none aok-align-center").find("span",class_="a-price aok-align-center reinventPricePriceToPayMargin priceToPay").find("span",class_="a-offscreen").get_text().replace(",",".")
        table=soup.find("table",id="productDetails_techSpec_section_1")
        
        for i in table.find_all("tr")[1:]:
            row_data=i.find_all("td")
            row=[j.text for j in row_data]
            
            product_details.append(row[0].replace("\n","").strip().replace("\u200e",""))
    


    finally:    
        if code==200:
            value_dict={"url":url,
                        "availibility":"available",
                        "title":title,
                       "img_link":img_link,
                       "price":price,
                       "product_detail":product_details}
            return value_dict
        else:
            return {"url":url,
                    "availbility":"not available"}

number_list=[float(i) for i in range(1,11)]
# header=["title","img_link","price","product_details"]
with open("main_dataset.json","w",encoding="utf-8") as f:
    start_time=time.time()
    for i,url in enumerate(df.url.values):
        
        json.dump(scrape(url),f)
        if i/100 in number_list:
            print(time.time()-start_time)

94.4457573890686
171.91106939315796
249.5796880722046
320.60138297080994
389.50621604919434
465.57878971099854
539.5571172237396
608.8807322978973
675.3535900115967
